In [26]:
import os
%pwd
new_directory = r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS"

# Change the current working directory
os.chdir(new_directory)

In [27]:
%pwd

'C:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS'

In [28]:
import os

os.environ['MLFLOW_TRACKING_URL'] = "https://dagshub.com/aditi-singh-21/Chest-Cancer-ML-OPS.mlflow/#/"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'aditi-singh-21'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'aditisingh2114@'


In [29]:
import tensorflow as tf

In [30]:
model = tf.keras.models.load_model(r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\artifacts\training\model.h5")

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_url : str
    image_size_params : list
    batch_size_params : int

In [32]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml,create_dir,save_json

In [33]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_dir([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\artifacts\training\model.h5",
            training_data=r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\artifacts\data_ingestion\Data\test",
            mlflow_url="https://dagshub.com/aditi-singh-21/Chest-Cancer-ML-OPS.mlflow/#/",
            all_params= self.params,
            image_size_params=self.params.IMAGE_SIZE,
            batch_size_params=self.params.BATCH_SIZE
        )
        return eval_config

In [34]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [35]:
class Evaluation:
    def __init__(self,config : EvaluationConfig):
        self.config = config
        
    def val_gen(self):
         
        datagenrator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )   
        
        dataflow_kwargs = dict(
            target_size=self.config.image_size_params[:-1],
            batch_size=self.config.batch_size_params,
            interpolation="bilinear"
        )
        
        valid_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenrator_kwargs
        )
        
        self.valid_gen = valid_data_gen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.val_gen()
        self.score = model.evaluate(self.valid_gen)
        self.save_score()
        
    def save_score(self):
        scores = {"loss" : self.score[0] , "accuracy" : self.score[1]}
        save_json(path=Path("scores.json"),data = scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss" : self.score[0] , "accuracy" : self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")
        
        
        

In [36]:
try:
    config = ConfigManager()
    eval_config = config.get_evaluation_config()
    eval=Evaluation(eval_config)
    eval.evaluation()
    eval.log_into_mlflow()
    
except Exception as e:
    raise e

path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml
[2024-09-01 17:33:50,742 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml successfully loaded]
path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml
[2024-09-01 17:33:50,746 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml successfully loaded]
[2024-09-01 17:33:50,748 INFO : common : Created Directory at : artifacts]
Found 267 images belonging to 4 classes.
17/17 [==============================] - 97s 6s/step - loss: 26.8377 - accuracy: 0.1798
[2024-09-01 17:35:28,886 INFO : common : json file saved at: scores.json]


2024/09/01 17:35:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-09-01 17:35:32,143 WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\aditi\AppData\Local\Temp\tmp1jj2vb4a\model\data\model\assets
[2024-09-01 17:35:33,355 INFO : builder_impl : Assets written to: C:\Users\aditi\AppData\Local\Temp\tmp1jj2vb4a\model\data\model\assets]


2024/09/01 17:35:48 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
